In [17]:
import pandas as pd

messages = pd.read_csv('messages.csv')
messages.head(20)

,messages,has_photo,has_video
0,Хай,0,0
1,Как сделать свой стиль ?,0,0
2,"Если скоро не будет , то удалю приложение нафиг",0,0
3,Окей,0,0
4,Не работает,0,0
5,Как сделать *был в сети недавно*,0,0
6,Привет,0,0
7,Салам,0,0
8,"Спасибо, помогло)",0,0
9,"Здравствуйте, не грузится список диалогов",1,0


In [5]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/msnthrp/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
russian_stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [18]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    return " ".join(tokens)

clean_messages = messages.copy()
clean_messages['messages'] = [preprocess_text(i) for i in messages['messages']]
clean_messages.head(20)

,messages,has_photo,has_video
0,хай,0,0
1,сделать свой стиль,0,0
2,скоро удалять приложение нафиг,0,0
3,окей,0,0
4,работать,0,0
5,сделать сеть недавно,0,0
6,привет,0,0
7,сало,0,0
8,спасибо помогать,0,0
9,здравствовать грузиться список диалог,1,0


In [12]:
clean_messages.to_csv('clean_messages.csv', index=False)

In [19]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(list(clean_messages['messages']))
X

<1634x2095 sparse matrix of type '<class 'numpy.float64'>'
	with 6888 stored elements in Compressed Sparse Row format>

In [27]:
for cl_i in range(2, 10):
    model = KMeans(n_clusters=cl_i)
    model.fit(X)
    print(f'{cl_i}: {model.inertia_}')

2: 1417.7641968976784
3: 1367.7406124444642
4: 1357.5611983063075
5: 1350.89650860098
6: 1294.436451244549
7: 1300.9748975841287
8: 1280.4283485032638
9: 1276.8405613663642


In [54]:
from sklearn.cluster import SpectralClustering
import collections

for cl_i in range(2, 30):
    print(f'\n-------- CLUSTERS: {cl_i} --------')
    model = SpectralClustering(n_clusters=cl_i)
    model.fit(X)
    cluster_count = collections.Counter(model.labels_)
    for lab_i in range(cl_i):
        cluster_messages = []
        for i in range(len(clean_messages['messages'])):
            if model.labels_[i] == lab_i:
                cluster_messages += clean_messages['messages'][i].split(' ')
        count = collections.Counter(cluster_messages)
        
        total = len(cluster_messages)
        print(f'\ncluster {lab_i}: words size = {total}')
        for word, cnt in count.items():
            if cnt > total * 0.1:
                print(f'{word} {cnt / total * 100:.2f}%\t\t', end='')
        print('')


-------- CLUSTERS: 2 --------

cluster 0: words size = 7756


cluster 1: words size = 64
спасибо 90.62%		

-------- CLUSTERS: 3 --------

cluster 0: words size = 7705


cluster 1: words size = 64
спасибо 90.62%		

cluster 2: words size = 51
привет 98.04%		

-------- CLUSTERS: 4 --------

cluster 0: words size = 7659


cluster 1: words size = 46
здравствовать 97.83%		

cluster 2: words size = 51
привет 98.04%		

cluster 3: words size = 64
спасибо 90.62%		

-------- CLUSTERS: 5 --------

cluster 0: words size = 7646


cluster 1: words size = 51
привет 98.04%		

cluster 2: words size = 46
здравствовать 97.83%		

cluster 3: words size = 64
спасибо 90.62%		

cluster 4: words size = 13
понятно 92.31%		

-------- CLUSTERS: 6 --------

cluster 0: words size = 7625


cluster 1: words size = 46
здравствовать 97.83%		

cluster 2: words size = 64
спасибо 90.62%		

cluster 3: words size = 13
понятно 92.31%		

cluster 4: words size = 51
привет 98.04%		

cluster 5: words size = 21
понимать 61.90%		с


cluster 0: words size = 906
сообщение 12.58%		

cluster 1: words size = 5127


cluster 2: words size = 51
привет 98.04%		

cluster 3: words size = 37
музыка 48.65%		слушать 10.81%		

cluster 4: words size = 34
спасибо 38.24%		большой 38.24%		

cluster 5: words size = 15
понимать 73.33%		спасибо 20.00%		

cluster 6: words size = 13
понятно 92.31%		

cluster 7: words size = 64
спасибо 90.62%		

cluster 8: words size = 13
ладно 61.54%		спасибо 23.08%		

cluster 9: words size = 10
почему 80.00%		?? 20.00%		

cluster 10: words size = 156
мочь 22.44%		

cluster 11: words size = 46
здравствовать 97.83%		

cluster 12: words size = 16
сделать 43.75%		сеть 37.50%		недавно 18.75%		

cluster 13: words size = 7
пожалуйста 71.43%		сделать 14.29%		😉 14.29%		

cluster 14: words size = 72
свой 22.22%		страница 12.50%		

cluster 15: words size = 71
это 40.85%		

cluster 16: words size = 733
приложение 15.01%		

cluster 17: words size = 255
вк 17.65%		

cluster 18: words size = 194
работать 13.92%		

--


cluster 0: words size = 5062


cluster 1: words size = 4
hi 100.00%		

cluster 2: words size = 10
почему 80.00%		?? 20.00%		

cluster 3: words size = 46
здравствовать 97.83%		

cluster 4: words size = 59
свой 23.73%		профиль 10.17%		зайти 10.17%		страница 13.56%		

cluster 5: words size = 34
спасибо 38.24%		большой 38.24%		

cluster 6: words size = 13
понятно 92.31%		

cluster 7: words size = 15
понимать 73.33%		спасибо 20.00%		

cluster 8: words size = 4
дело 100.00%		

cluster 9: words size = 13
ладно 61.54%		спасибо 23.08%		

cluster 10: words size = 51
привет 98.04%		

cluster 11: words size = 4
ку 100.00%		

cluster 12: words size = 7
пожалуйста 71.43%		сделать 14.29%		😉 14.29%		

cluster 13: words size = 15
жаль 46.67%		

cluster 14: words size = 24
знать 33.33%		

cluster 15: words size = 64
спасибо 90.62%		

cluster 16: words size = 37
музыка 48.65%		слушать 10.81%		

cluster 17: words size = 273
вк 17.22%		

cluster 18: words size = 99
мочь 27.27%		

cluster 19: words size = 